# Análise de gastos da UFCG em relação ao seu gasto

## Importando bibliotecas e preparando dados

In [26]:
# Importações
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [27]:
orcamento_completo = pd.read_csv('../consulta_siop_orcamento_ufcg.csv')
orcamento_objetivo = orcamento_completo.iloc[:, 18:]
orcamento_objetivo['ano'] = orcamento_completo['ano']
orcamento_objetivo

,objetivo,projeto_de_lei,dotacao_inicial,dotacao_atual,empenhado,liquidado,pago,ano
0,[valor não detalhado],0,7.686.431,14.586.431,0,0,0,2012
1,[valor não detalhado],0,22.956.997,22.956.997,0,0,0,2012
2,[valor não detalhado],0,38.666.570,38.666.570,0,0,0,2012
3,[valor não detalhado],21.584.508,0,0,0,0,0,2012
4,[valor não detalhado],38.666.570,0,0,0,0,0,2012
...,...,...,...,...,...,...,...,...
4958,[valor não detalhado],0,13.000,13.000,0,0,0,2023
4959,[valor não detalhado],13.000,0,0,0,0,0,2023
4960,[valor não detalhado],0,68.000,68.000,0,0,0,2023
4961,[valor não detalhado],68.000,0,0,0,0,0,2023


## Tratando os dados

Nosso objetivo é refinar os dados que nós já temos, para termos informações mais fáceis a serem analisadas. Com este intuito, será utilizado uma série de computações para a criação de uma tabela dinâmica em que teremos informações específicas de cada ano para cada objetivo.

### Cuidados básicos

Tratando o DataFrame para padronizar e possibilitar a análise.

In [28]:
objetivo_copia: pd.DataFrame = orcamento_objetivo.copy(deep=True)

for coluna in objetivo_copia.columns[1:-1]:
    objetivo_copia[coluna] = objetivo_copia[coluna].map(lambda value: value.replace(".",""))
    objetivo_copia[coluna] = objetivo_copia[coluna].astype('int64')

objetivo_copia


,objetivo,projeto_de_lei,dotacao_inicial,dotacao_atual,empenhado,liquidado,pago,ano
0,[valor não detalhado],0,7686431,14586431,0,0,0,2012
1,[valor não detalhado],0,22956997,22956997,0,0,0,2012
2,[valor não detalhado],0,38666570,38666570,0,0,0,2012
3,[valor não detalhado],21584508,0,0,0,0,0,2012
4,[valor não detalhado],38666570,0,0,0,0,0,2012
...,...,...,...,...,...,...,...,...
4958,[valor não detalhado],0,13000,13000,0,0,0,2023
4959,[valor não detalhado],13000,0,0,0,0,0,2023
4960,[valor não detalhado],0,68000,68000,0,0,0,2023
4961,[valor não detalhado],68000,0,0,0,0,0,2023


### Agrupar informações

Temos muitas informações separadas, ou seja, temos mais de quatro mil linhas mas não temos visão do todo. Essa sessão será para criar uma tabela que sintetize a tabela completa em uma partição muito menor, que possibilite a identificação de comportamentos em escala de anos.

In [29]:
monetario = [
    'projeto_de_lei', 
    'dotacao_inicial', 
    'dotacao_atual', 
    'empenhado', 
    'liquidado', 
    'pago'
]

alvo = [
    'objetivo',
    'ano'
]

In [30]:
objetivo_informacoes = objetivo_copia.groupby(alvo)[monetario].agg([
    sum, len, max, min, lambda col_monetaria: col_monetaria.mean()
])

objetivo_informacoes.rename(columns={'<lambda_0>':'media', 
                                     'sum': 'soma', 
                                     'len': 'quantidade de gastos', 
                                     'min': 'minimo',
                                     'max': 'maximo'}, inplace=True)
objetivo_informacoes

projeto_de_lei  \
                                                                  soma   
objetivo                                           ano                   
0582 - Expandir, interiorizar, democratizar e q... 2014        1598204   
                                                   2015        1511577   
0597 - Promover, em articulação com os sistemas... 2012         229892   
                                                   2013        1360240   
                                                   2014        1704881   
                                                   2015         642701   
0803 - Apoiar a formação de pessoal qualificado... 2012         461369   
                                                   2013        2324165   
                                                   2014        2011276   
                                                   2015        1252583   
0841 - Ampliar o acesso à educação superior com... 2012       74484103   
                                                   2013       84182086   
                                                   2014       93645202   
                                                   2015      107301602   
1009 - Ampliar o acesso à educação profissional... 2016        1903044   
                                                   2017        1541230   
                                                   2018        1229204   
                                                   2019        1659645   
1010 - Ampliar o acesso à educação superior de ... 2016      110671031   
                                                   2017      102848784   
                                                   2018       87532645   
                                                   2019       89416257   
1204 - Ampliar o acesso à educação profissional... 2020         982507   
                                                   2021         959417   
                                                   2022        1704567   
                                                   2023        1661362   
1237 - Fomentar a formação de pessoal qualifica... 2020       55953608   
                                                   2021       32587658   
                                                   2022       77579609   
                                                   2023       72887646   
[valor não detalhado]                              2012      304127438   
                                                   2013      315468518   
                                                   2014      381347718   
                                                   2015      422531400   
                                                   2016      478443148   
                                                   2017      542525837   
                                                   2018      564164581   
                                                   2019      591671792   
                                                   2020      578404832   
                                                   2021      366415747   
                                                   2022      664228115   
                                                   2023      679352320   

                                                                              \
                                                        quantidade de gastos   
objetivo                                           ano                         
0582 - Expandir, interiorizar, democratizar e q... 2014                   40   
                                                   2015                   41   
0597 - Promover, em articulação com os sistemas... 2012                    4   
                                                   2013                   16   
                                                   2014                    8   
                                                   2015                    7   
0803 

### Filtração dos dados em Ensino Superior e Formação profissional

O objetivo desta etapa é produzir uma nova tabela dinâmica que contenha os gastos em objetivos traduzidos para "Ensino Superior" e "Formação Profissional". A partir desta nova tabela, será filtrado os dados em tabelas menores em que teremos informações específicas de gastos do ensino superior e de formação profissional.

In [129]:
objetivo_geral = objetivo_copia.copy(deep=True)
objetivo_geral.drop(objetivo_geral[objetivo_geral.objetivo == '[valor não detalhado]'].index, inplace=True)
objetivo_geral

,objetivo,projeto_de_lei,dotacao_inicial,dotacao_atual,empenhado,liquidado,pago,ano
32,"0597 - Promover, em articulação com os sistema...",0,227392,227392,0,0,0,2012
33,"0597 - Promover, em articulação com os sistema...",227392,0,0,0,0,0,2012
34,"0597 - Promover, em articulação com os sistema...",0,2500,2500,0,0,0,2012
35,"0597 - Promover, em articulação com os sistema...",2500,0,0,0,0,0,2012
36,0803 - Apoiar a formação de pessoal qualificad...,0,375208,375208,0,0,0,2012
...,...,...,...,...,...,...,...,...
4953,1237 - Fomentar a formação de pessoal qualific...,0,0,0,2071551,2068767,2068767,2023
4954,1237 - Fomentar a formação de pessoal qualific...,0,1500000,0,0,0,0,2023
4955,1237 - Fomentar a formação de pessoal qualific...,0,0,4087661,0,0,0,2023
4956,1237 - Fomentar a formação de pessoal qualific...,0,3400000,3400000,0,0,0,2023


In [130]:
objetivo_geral.objetivo.replace('0582 - Expandir, interiorizar, democratizar e qualificar a oferta...', 'ensino_superior', inplace=True)
objetivo_geral.objetivo.replace('0597 - Promover, em articulação com os sistemas de ensino estaduais...', 'ensino_basico', inplace=True)
objetivo_geral.objetivo.replace('0803 - Apoiar a formação de pessoal qualificado em nível superior...', 'formacao_profissional', inplace=True)
objetivo_geral.objetivo.replace('0841 - Ampliar o acesso à educação superior com condições de...', 'ensino_superior', inplace=True)
objetivo_geral.objetivo.replace('1009 - Ampliar o acesso à educação profissional e tecnológica de...', 'formacao_profissional', inplace=True)
objetivo_geral.objetivo.replace('1010 - Ampliar o acesso à educação superior de qualidade, na...', 'ensino_superior', inplace=True)
objetivo_geral.objetivo.replace('1204 - Ampliar o acesso à educação profissional e tecnológica, em...', 'formacao_profissional', inplace=True)
objetivo_geral.objetivo.replace('1237 - Fomentar a formação de pessoal qualificado, fortalecendo a...', 'formacao_profissional', inplace=True)
objetivo_geral.objetivo.unique()

array(['ensino_basico', 'formacao_profissional', 'ensino_superior'],
      dtype=object)

In [131]:
colunas_alvo_soma = [
    ('projeto_de_lei','soma'),
    ('dotacao_inicial','soma'),
    ('dotacao_atual','soma'),
    ('empenhado','soma'),
    ('liquidado','soma'),
    ('pago','soma')
]

In [132]:
objetivo_geral_comparacoes = objetivo_geral.groupby(alvo)[monetario].agg([
    sum, len, max, min, lambda col_monetaria: col_monetaria.mean()
])

objetivo_geral_comparacoes.rename(columns={'<lambda_0>':'media', 
                                     'sum': 'soma', 
                                     'len': 'quantidade de gastos', 
                                     'min': 'minimo',
                                     'max': 'maximo'}, inplace=True)

In [133]:
objetivo_geral_comparacoes

projeto_de_lei                                 \
                                     soma quantidade de gastos    maximo   
objetivo              ano                                                  
ensino_basico         2012         229892                    4    227392   
                      2013        1360240                   16   1360240   
                      2014        1704881                    8   1704881   
                      2015         642701                    7    642701   
ensino_superior       2012       74484103                  270  16264609   
                      2013       84182086                  371  14303080   
                      2014       95243406                  302  19500000   
                      2015      108813179                  304  25000000   
                      2016      110671031                  259  22401000   
                      2017      102848784                  223  18992524   
                      2018       87532645                  212  30000000   
                      2019       89416257                  234  28000000   
formacao_profissional 2012         461369                   32    181440   
                      2013        2324165                   41   1000000   
                      2014        2011276                   35   1300000   
                      2015        1252583                   59    492935   
                      2016        1903044                   51    385000   
                      2017        1541230                   53    355000   
                      2018        1229204                   40    398000   
                      2019        1659645                   46    398000   
                      2020       56936115                  240  16576000   
                      2021       33547075                  218   6464492   
                      2022       79284176                  256  20340000   
                      2023       74549008                  156  18010819   

                                                 dotacao_inicial  \
                           minimo          media            soma   
objetivo              ano                                          
ensino_basico         2012      0   57473.000000          229892   
                      2013      0   85015.000000         1360240   
                      2014      0  213110.125000         1104881   
                      2015      0   91814.428571          642701   
ensino_superior       2012      0  275867.048148        79184103   
                      2013      0  226905.892183        86582086   
                      2014      0  315375.516556        97943406   
                      2015      0  357938.088816       110613179   
                      2016      0  427301.277992       127831193   
                      2017      0  461205.309417       108998784   
                      2018      0  412889.834906        90528501   
                      2019      0  382120.756410       117666663   
formacao_profissional 2012      0   14417.781250          461369   
                      2013      0   56686.951220         2324165   
                      2014      0   57465.028571         2011276   
                      2015      0   21230.220339         1252583   
                      2016      0   37314.588235         1903044   
                      2017      0   29079.811321         1541230   
                      2018      0   30730.100000         1229204   
                      2019      0   36079.239130         1659645   
                      2020      0  237233.812500        56810415   
                      2021      0  153885.665138        31501687   
                      2022      0  309703.812500        88258403   
                      2023      0  477878.256410        77054177   

                                                                  \
                           quantidade de gastos    maximo minimo   
o

In [135]:
objetivo_ensino_superior = objetivo_geral_comparacoes.iloc[4:12]
objetivo_ensino_superior

projeto_de_lei                                        \
                               soma quantidade de gastos    maximo minimo   
objetivo        ano                                                         
ensino_superior 2012       74484103                  270  16264609      0   
                2013       84182086                  371  14303080      0   
                2014       95243406                  302  19500000      0   
                2015      108813179                  304  25000000      0   
                2016      110671031                  259  22401000      0   
                2017      102848784                  223  18992524      0   
                2018       87532645                  212  30000000      0   
                2019       89416257                  234  28000000      0   

                                    dotacao_inicial                       \
                              media            soma quantidade de gastos   
objetivo        ano                                                        
ensino_superior 2012  275867.048148        79184103                  270   
                2013  226905.892183        86582086                  371   
                2014  315375.516556        97943406                  302   
                2015  357938.088816       110613179                  304   
                2016  427301.277992       127831193                  259   
                2017  461205.309417       108998784                  223   
                2018  412889.834906        90528501                  212   
                2019  382120.756410       117666663                  234   

                                                      ... liquidado  \
                        maximo minimo          media  ...      soma   
objetivo        ano                                   ...             
ensino_superior 2012  31117535      0  293274.455556  ...  49573149   
                2013  34211912      0  233374.894879  ...  63170618   
                2014  41014888      0  324315.913907  ...  64220981   
                2015  48458138      0  363859.141447  ...  70505332   
                2016  51934094      0  493556.729730  ...  69885656   
                2017  49988767      0  488783.784753  ...  60327327   
                2018  56748411      0  427021.231132  ...  54572959   
                2019  56744247      0  502848.987179  ...  50076838   

                                                                           \
                     quantidade de gastos    maximo minimo          media   
objetivo        ano                                                         
ensino_superior 2012                  270   8215761      0  183604.255556   
                2013                  371  14309592      0  170271.207547   
                2014                  302  14972325      0  212652.254967   
                2015                  304  18872392      0  231925.434211   
                2016                  259  17705662      0  269828.787645   
                2017                  223  16362275      0  270526.130045   
                2018                  212  15389881      0  257419.617925   
                2019                  234  10991901      0  214003.581197   

                          pago                                        \
                          soma quantidade de gastos    maximo minimo   
objetivo        ano                                                    
ensino_superior 2012  48063918                  270   8011360      0   
                2013  60636000                  371  14240406      0   
                2014  63203838                  302  14844818      0   
                2015  64216241                  304  18162988      0   
                2016  69016877                  259  17705662      0   
                2017  59389224                  223  15643632      0   
                2018  54012312                  212  14963

In [136]:
objetivo_ensino_superior = objetivo_ensino_superior.reset_index().iloc[:,1:]
objetivo_ensino_superior.index = objetivo_ensino_superior.iloc[:,0]
objetivo_ensino_superior = objetivo_ensino_superior.iloc[:,1:]
objetivo_ensino_superior.index.rename('ano', inplace=True)
objetivo_ensino_superior

projeto_de_lei                                                       \
               soma quantidade de gastos    maximo minimo          media   
ano                                                                        
2012       74484103                  270  16264609      0  275867.048148   
2013       84182086                  371  14303080      0  226905.892183   
2014       95243406                  302  19500000      0  315375.516556   
2015      108813179                  304  25000000      0  357938.088816   
2016      110671031                  259  22401000      0  427301.277992   
2017      102848784                  223  18992524      0  461205.309417   
2018       87532645                  212  30000000      0  412889.834906   
2019       89416257                  234  28000000      0  382120.756410   

     dotacao_inicial                                                       \
                soma quantidade de gastos    maximo minimo          media   
ano                                                                         
2012        79184103                  270  31117535      0  293274.455556   
2013        86582086                  371  34211912      0  233374.894879   
2014        97943406                  302  41014888      0  324315.913907   
2015       110613179                  304  48458138      0  363859.141447   
2016       127831193                  259  51934094      0  493556.729730   
2017       108998784                  223  49988767      0  488783.784753   
2018        90528501                  212  56748411      0  427021.231132   
2019       117666663                  234  56744247      0  502848.987179   

      ... liquidado                                                       \
      ...      soma quantidade de gastos    maximo minimo          media   
ano   ...                                                                  
2012  ...  49573149                  270   8215761      0  183604.255556   
2013  ...  63170618                  371  14309592      0  170271.207547   
2014  ...  64220981                  302  14972325      0  212652.254967   
2015  ...  70505332                  304  18872392      0  231925.434211   
2016  ...  69885656                  259  17705662      0  269828.787645   
2017  ...  60327327                  223  16362275      0  270526.130045   
2018  ...  54572959                  212  15389881      0  257419.617925   
2019  ...  50076838                  234  10991901      0  214003.581197   

          pago                                                       
          soma quantidade de gastos    maximo minimo          media  
ano                                                                  
2012  48063918                  270   8011360      0  178014.511111  
2013  60636000                  371  14240406      0  163439.353100  
2014  63203838                  302  14844818      0  209284.231788  
2015  64216241                  304  18162988      0  211237.634868  
2016  69016877                  259  17705662      0  266474.428571  
2017  59389224                  223  15643632      0  266319.390135  
2018  54012312                  212  14963179      0  254775.056604  
2019  49672665                  234  10991901      0  212276.346154  

[8 rows x 30 columns]

In [137]:
objetivo_formacao_profissional = objetivo_geral_comparacoes.iloc[12:]
objetivo_formacao_profissional

projeto_de_lei                                 \
                                     soma quantidade de gastos    maximo   
objetivo              ano                                                  
formacao_profissional 2012         461369                   32    181440   
                      2013        2324165                   41   1000000   
                      2014        2011276                   35   1300000   
                      2015        1252583                   59    492935   
                      2016        1903044                   51    385000   
                      2017        1541230                   53    355000   
                      2018        1229204                   40    398000   
                      2019        1659645                   46    398000   
                      2020       56936115                  240  16576000   
                      2021       33547075                  218   6464492   
                      2022       79284176                  256  20340000   
                      2023       74549008                  156  18010819   

                                                 dotacao_inicial  \
                           minimo          media            soma   
objetivo              ano                                          
formacao_profissional 2012      0   14417.781250          461369   
                      2013      0   56686.951220         2324165   
                      2014      0   57465.028571         2011276   
                      2015      0   21230.220339         1252583   
                      2016      0   37314.588235         1903044   
                      2017      0   29079.811321         1541230   
                      2018      0   30730.100000         1229204   
                      2019      0   36079.239130         1659645   
                      2020      0  237233.812500        56810415   
                      2021      0  153885.665138        31501687   
                      2022      0  309703.812500        88258403   
                      2023      0  477878.256410        77054177   

                                                                  \
                           quantidade de gastos    maximo minimo   
objetivo              ano                                          
formacao_profissional 2012                   32    375208      0   
                      2013                   41   1075000      0   
                      2014                   35   1300000      0   
                      2015                   59    492935      0   
                      2016                   51    843788      0   
                      2017                   53    795145      0   
                      2018                   40    905204      0   
                      2019                   46   1295595      0   
                      2020                  240  32487620      0   
                      2021                  218  16757456      0   
                      2022                  256  49268772      0   
                      2023                  156  42782598      0   

                                           ... liquidado                       \
                                    media  ...      soma quantidade de gastos   
objetivo              ano                  ...                                  
formacao_profissional 2012   14417.781250  ...    270008                   32   
                      2013   56686.951220  ...    107149                   41   
                      2014   57465.028571  ...    251226                   35   
                      2015   21230.220339  ...    208791                   59   
                      2016   37314.588235  ...    778554                   51   
                      2017   29079.811321  ...    517989                   53   
                      2018   30730.100000  ...   1009925                   40   
                      2019   36079

In [138]:
objetivo_formacao_profissional = objetivo_formacao_profissional.reset_index().iloc[:,1:]
objetivo_formacao_profissional.index = objetivo_formacao_profissional.iloc[:,0]
objetivo_formacao_profissional = objetivo_formacao_profissional.iloc[:,1:]
objetivo_formacao_profissional.index.rename('ano', inplace=True)
objetivo_formacao_profissional

projeto_de_lei                                                       \
               soma quantidade de gastos    maximo minimo          media   
ano                                                                        
2012         461369                   32    181440      0   14417.781250   
2013        2324165                   41   1000000      0   56686.951220   
2014        2011276                   35   1300000      0   57465.028571   
2015        1252583                   59    492935      0   21230.220339   
2016        1903044                   51    385000      0   37314.588235   
2017        1541230                   53    355000      0   29079.811321   
2018        1229204                   40    398000      0   30730.100000   
2019        1659645                   46    398000      0   36079.239130   
2020       56936115                  240  16576000      0  237233.812500   
2021       33547075                  218   6464492      0  153885.665138   
2022       79284176                  256  20340000      0  309703.812500   
2023       74549008                  156  18010819      0  477878.256410   

     dotacao_inicial                                                       \
                soma quantidade de gastos    maximo minimo          media   
ano                                                                         
2012          461369                   32    375208      0   14417.781250   
2013         2324165                   41   1075000      0   56686.951220   
2014         2011276                   35   1300000      0   57465.028571   
2015         1252583                   59    492935      0   21230.220339   
2016         1903044                   51    843788      0   37314.588235   
2017         1541230                   53    795145      0   29079.811321   
2018         1229204                   40    905204      0   30730.100000   
2019         1659645                   46   1295595      0   36079.239130   
2020        56810415                  240  32487620      0  236710.062500   
2021        31501687                  218  16757456      0  144503.151376   
2022        88258403                  256  49268772      0  344759.386719   
2023        77054177                  156  42782598      0  493937.032051   

      ... liquidado                                                       \
      ...      soma quantidade de gastos    maximo minimo          media   
ano   ...                                                                  
2012  ...    270008                   32    165890      0    8437.750000   
2013  ...    107149                   41     45950      0    2613.390244   
2014  ...    251226                   35    188400      0    7177.885714   
2015  ...    208791                   59    151200      0    3538.830508   
2016  ...    778554                   51    217712      0   15265.764706   
2017  ...    517989                   53     99897      0    9773.377358   
2018  ...   1009925                   40    266170      0   25248.125000   
2019  ...    260009                   46     33418      0    5652.369565   
2020  ...  43463846                  240   7436508      0  181099.358333   
2021  ...  45160187                  218  12554591      0  207156.821101   
2022  ...  61502441                  256  17384365      0  240243.910156   
2023  ...  22696458                  156   6205710      0  145490.115385   

          pago                                                       
          soma quantidade de gastos    maximo minimo          media  
ano                                                                  
2012    246000                   32    163010      0    7687.500000  
2013    106789                   41     45590      0    2604.609756  
2014    242401                   35    187600      0    6925.742857  
2015    191166                   59    145200      0    3240.101695  
2016    695738                   51    217712      0   13641.921569  
2017   

### Criação de tabelas específicas

Essa sessão será justamente para a criação de tabelas mais específicas, com o fim de gerar gráficos de linhas para a identificação de possíveis comportamentos ou a falta deles. Note que cada tabela representa um objetivo, por isso o nome da variável está separada e pareada com ele.

In [39]:
objetivo_0582 = objetivo_informacoes.reset_index().iloc[:2,1:]
objetivo_0582.index = objetivo_0582.iloc[:,0]
objetivo_0582.drop(columns=['ano'], inplace=True)
objetivo_0597 = objetivo_informacoes.reset_index().iloc[2:6,1:]
objetivo_0597.index = objetivo_0597.iloc[:,0]
objetivo_0597.drop(columns=['ano'], inplace=True)
objetivo_0803  = objetivo_informacoes.reset_index().iloc[6:10,1:]
objetivo_0803.index = objetivo_0803.iloc[:,0]
objetivo_0803.drop(columns=['ano'], inplace=True)
objetivo_0841 = objetivo_informacoes.reset_index().iloc[10:14,1:]
objetivo_0841.index = objetivo_0841.iloc[:,0]
objetivo_0841.drop(columns=['ano'], inplace=True)
objetivo_1009  = objetivo_informacoes.reset_index().iloc[14:18,1:]
objetivo_1009.index = objetivo_1009.iloc[:,0]
objetivo_1009.drop(columns=['ano'], inplace=True)
objetivo_1010  = objetivo_informacoes.reset_index().iloc[18:22,1:]
objetivo_1010.index = objetivo_1010.iloc[:,0]
objetivo_1010.drop(columns=['ano'], inplace=True)
objetivo_1204 = objetivo_informacoes.reset_index().iloc[22:26,1:]
objetivo_1204.index = objetivo_1204.iloc[:,0]
objetivo_1204.drop(columns=['ano'], inplace=True)
objetivo_1237 = objetivo_informacoes.reset_index().iloc[26:30,1:]
objetivo_1237.index = objetivo_1237.iloc[:,0]
objetivo_1237.drop(columns=['ano'], inplace=True)
objetivo_nao_detalhado = objetivo_informacoes.reset_index().iloc[30:,1:]
objetivo_nao_detalhado.index = objetivo_nao_detalhado.iloc[:,0]
objetivo_nao_detalhado.drop(columns=['ano'], inplace=True)


C:\Users\Gabriel\AppData\Local\Temp\ipykernel_27108\1602861820.py:5: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  objetivo_0582.drop(columns=['ano'], inplace=True)
C:\Users\Gabriel\AppData\Local\Temp\ipykernel_27108\1602861820.py:8: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  objetivo_0597.drop(columns=['ano'], inplace=True)
C:\Users\Gabriel\AppData\Local\Temp\ipykernel_27108\1602861820.py:11: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  objetivo_0803.drop(columns=['ano'], inplace=True)
C:\Users\Gabriel\AppData\Local\Temp\ipykernel_27108\1602861820.py:14: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  objetivo_0841.drop(columns=['ano'], inplace=True)
C:\Users\Gabriel\AppData\Local\Temp\ipykernel_27108\1602861820.py:17: Perf

# Analisando os dados